In [1]:
from astroquery.skyview import SkyView
from astroquery.mast import Observations
from astroquery.vizier import Vizier
from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import cv2
import numpy as np
from astropy.io import fits
import os
import pandas as pd
import requests
import json

In [2]:
# # List available surveys
# surveys = SkyView.list_surveys()
# print(surveys)

In [3]:
# Define the coordinates for the Eagle Nebula Pillars of Creation
coords = SkyCoord('18h18m48s -13d49m00s', frame='icrs')
# Track the name of the object
object_name = 'Eagle Nebula Pillars of Creation'

In [4]:
# # Define the coordinates for the M87 Black Hole
# coords = SkyCoord('12h30m49.42338s +12°23m28.0439s', frame='icrs')
# # Track the name of the object
# object_name = 'M87 Black Hole'

In [5]:
# # Define the coordinates for the Crab Nebula
# coords = SkyCoord('05h34m31.94s +22d00m52.2s', frame='icrs')
# # Track the name of the object
# object_name = 'Crab Nebula'

In [6]:
# # Define the coordinates for the Orion Nebula
# coords = SkyCoord('05h35m17.3s -05d23m28s', frame='icrs')
# # Track the name of the object
# object_name = 'Orion Nebula'

In [7]:
# Fetch an image from SkyView
# image_list = SkyView.get_images(position=coords, survey=['DSS'], radius=0.1 * u.deg)
# image_list = SkyView.get_images(position=coords, survey=['DSS'], radius=0.25 * u.deg)
# image_list = SkyView.get_images(position=coords, survey=['DSS'], radius=0.5 * u.deg)
# image_list = SkyView.get_images(position=coords, survey=['DSS'], radius=1 * u.deg)

image_list = SkyView.get_images(position=coords, survey=['DSS1 Blue'], radius=0.25 * u.deg)
# image_list = SkyView.get_images(position=coords, survey=['DSS1 Red'], radius=0.25 * u.deg)
# image_list = SkyView.get_images(position=coords, survey=['DSS2 Red'], radius=0.25 * u.deg)

image_hdu = image_list[0][0]
image = image_list[0][0].data

In [8]:
# Create a folder for the object with the name of the object with underscores instead of spaces
object_name = object_name.replace(' ', '_')
os.makedirs(f'images/{object_name}', exist_ok=True)

In [9]:
# # Save the image as a FITS file
# fits_file_path = f'images/{object_name}/{object_name}.fits'
# hdu = fits.PrimaryHDU(image)
# hdul = fits.HDUList([hdu])
# hdul.writeto(fits_file_path, overwrite=True)

In [10]:
# # Display the image
# plt.imshow(image, cmap='gray')

In [11]:
# # Download the image in all available color maps to the object folder
# for cmap in plt.colormaps():
#     plt.imsave(f'images/{object_name}/{object_name}_{cmap}.png', image, cmap=cmap)

In [12]:
# # Visualize al color maps in a single image
# fig, ax = plt.subplots(1, 1, figsize=(12, 12))
# for i, cmap in enumerate(plt.colormaps()):
#     ax.imshow(image, cmap=cmap)
#     ax.set_title(cmap)
#     plt.axis('off')
#     plt.savefig(f'images/{object_name}/All_cmaps__{object_name}.png')

In [ ]:
image_hdu.header

In [ ]:
# Extract WCS information
wcs = WCS(image_hdu.header)

# Display the image with WCS projection and grid
fig = plt.figure(figsize=(10, 10), dpi=600)
ax = fig.add_subplot(111, projection=wcs)
ax.imshow(image_hdu.data, cmap='gray', origin='lower')
ax.set_title(f'{object_name}')
ax.set_xlabel('RA')
ax.set_ylabel('Dec')
ax.grid(color='white', ls='dotted')


# # Display the image without WCS projection
# plt.imshow(image, cmap='gray')
# plt.title('Sky Image')
# plt.xlabel('RA')
# plt.ylabel('Dec')
# plt.show()

In [ ]:
# Display the image with WCS projection and grid
fig = plt.figure(figsize=(14, 14), dpi=600)
ax = fig.add_subplot(111, projection=wcs)
ax.imshow(image_hdu.data, cmap='twilight_shifted_r', origin='lower')
ax.set_title(f'{object_name}')
ax.set_xlabel('RA')
ax.set_ylabel('Dec')
ax.grid(color='white', ls='dotted')

In [52]:
# Fetch star data from Vizier using the 2MASS catalog
v = Vizier(columns=['*'])
v.ROW_LIMIT = -1
catalog_list = v.query_region(coords, radius=0.1 * u.deg, catalog='II/246')
catalog = catalog_list[0]

# Convert the table to a pandas DataFrame for easier manipulation
catalog = catalog.to_pandas()
print(catalog)

# Save the catalog as a CSV file
catalog.to_csv(f'images/{object_name}/2MASS_catalog_{object_name}.csv', index=False)

         RAJ2000    DEJ2000            _2MASS       Jmag  e_Jmag    Hmag  \
0     274.706481 -13.908632  18184955-1354310  16.733000     NaN  15.207   
1     274.705929 -13.906877  18184942-1354247  14.692000   0.050  12.964   
2     274.697386 -13.915481  18184737-1354557  16.327999   0.127  15.377   
3     274.697667 -13.911509  18184744-1354414  16.341999     NaN  15.172   
4     274.706649 -13.911582  18184959-1354416  16.177000     NaN  14.480   
...          ...        ...               ...        ...     ...     ...   
3751  274.679283 -13.726274  18184302-1343345  17.229000     NaN  15.923   
3752  274.677543 -13.721797  18184261-1343184  15.626000     NaN  15.340   
3753  274.678865 -13.723316  18184292-1343239  15.265000   0.056  13.754   
3754  274.679030 -13.719441  18184296-1343099  12.673000   0.028  11.674   
3755  274.689558 -13.717358  18184549-1343024  12.571000   0.026  12.113   

      e_Hmag    Kmag  e_Kmag Qflg  Rflg  Bflg Cflg  Xflg  Aflg  
0      0.122  13.962  

In [61]:
# Finding the pixel coordinates of a star in the image

from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

# Extract WCS information from image
wcs = WCS(image_hdu.header)

print('*' * 10)

print("Testing with manually translated coordinates: ")
# 18184798-1354388 = 18h18m47.98s -13d54m38.8s
# c = SkyCoord("18h18m47.98s -13d54m38.8s", frame=ICRS)

# 18184744-1354414 = 18h18m47s -13d54m41s
# c = SkyCoord("18h18m47.44s -13d54m41.4s", frame=ICRS)

# 18190221-1353284 = 18h19m02.21s -13d53m28.4s
c = SkyCoord("18h19m02.21s -13d53m28.4s", frame=ICRS)

pixel_coords = wcs.world_to_pixel(c)
print('Pixel Coords:', pixel_coords)

# Function that takes a wcs object and returns an array of the range of ICRS coordinates in the image
def getCoordRangeFromPixels(wcs):

    x_dim = wcs.pixel_shape[0] # May need to swap x and y dim! (but I think it's right...)
    y_dim = wcs.pixel_shape[1]

    coord_range = {}

    coord_range['lower_left'] = wcs.all_pix2world([0], [0], 1)
    coord_range['lower_right'] = wcs.all_pix2world([x_dim], [0], 1)
    coord_range['upper_left'] = wcs.all_pix2world([0], [y_dim], 1)
    coord_range['upper_right'] = wcs.all_pix2world([x_dim], [y_dim], 1)
    
    return coord_range


range = getCoordRangeFromPixels(wcs)
print('RANGE:', range)

print('*' * 10)


# x_min = range['lower_left'][0]
# print('x_min:', x_min)
# x_max = range['lower_right'][0]
# print('x_max:', x_max)
# y_min = range['lower_left'][1]
# print('y_min:', y_min)
# y_max = range['upper_left'][1]
# print('y_max:', y_max)

# NOTE: Max and min are reversed for some reason.. orientation of image in coord system...?

x_max = range['lower_left'][0]
x_min = range['lower_right'][0]
y_max = range['lower_left'][1]
y_min = range['upper_left'][1]


def getStarsInImage(wcs, catalog):

    stars_in_image = []
    
    for star in catalog.iterrows(): 

        print('STAR:\n', star[1][0]) # REJ2000
        print('STAR:\n', star[1][1]) # DEJ2000

        rej = star[1][0]
        dej = star[1][1]

        if rej < x_max and rej > x_min: 

            if dej < y_max and dej > y_min: 

                # Then star is within bounds of image! Add it to a list of stars in the image
                stars_in_image += star

    return stars_in_image


getStarsInImage(wcs, catalog)



**********
Testing with manually translated coordinates: 
Pixel Coords: (array(115.00994688), array(104.78579365))
RANGE: {'lower_left': [array([274.95858419]), array([-14.06739415])], 'lower_right': [array([274.44313395]), array([-14.06739597])], 'upper_left': [array([274.95802983]), array([-13.56740241])], 'upper_right': [array([274.44368464]), array([-13.56740416])]}
**********
x_min: [274.95858419]
x_max: [274.44313395]
y_min: [-14.06739415]
y_max: [-13.56740241]
STAR:
 274.706481
STAR:
 -13.908632
STAR:
 274.705929
STAR:
 -13.906877
STAR:
 274.697386
STAR:
 -13.915481
STAR:
 274.697667
STAR:
 -13.911509
STAR:
 274.706649
STAR:
 -13.911582
STAR:
 274.706807
STAR:
 -13.910513
STAR:
 274.70812
STAR:
 -13.915083
STAR:
 274.705187
STAR:
 -13.916469
STAR:
 274.709242
STAR:
 -13.910283
STAR:
 274.708212
STAR:
 -13.912372
STAR:
 274.705313
STAR:
 -13.912054
STAR:
 274.705153
STAR:
 -13.904785
STAR:
 274.700506
STAR:
 -13.906254
STAR:
 274.699919
STAR:
 -13.910779
STAR:
 274.702311
STAR:
 

C:\Users\shoulau\AppData\Local\Temp\ipykernel_16008\766459410.py:61: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print('STAR:\n', star[1][0]) # REJ2000
C:\Users\shoulau\AppData\Local\Temp\ipykernel_16008\766459410.py:62: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print('STAR:\n', star[1][1]) # DEJ2000


STAR:
 274.673896
STAR:
 -13.812975
STAR:
 274.662768
STAR:
 -13.820352
STAR:
 274.673818
STAR:
 -13.820964
STAR:
 274.667762
STAR:
 -13.820629
STAR:
 274.669879
STAR:
 -13.819479
STAR:
 274.674859
STAR:
 -13.820986
STAR:
 274.668639
STAR:
 -13.814623
STAR:
 274.675095
STAR:
 -13.814061
STAR:
 274.669091
STAR:
 -13.827062
STAR:
 274.675104
STAR:
 -13.828447
STAR:
 274.668016
STAR:
 -13.829662
STAR:
 274.668697
STAR:
 -13.831712
STAR:
 274.67469
STAR:
 -13.816636
STAR:
 274.670584
STAR:
 -13.825723
STAR:
 274.670382
STAR:
 -13.832476
STAR:
 274.698034
STAR:
 -13.820145
STAR:
 274.693283
STAR:
 -13.822147
STAR:
 274.693638
STAR:
 -13.818611
STAR:
 274.68942
STAR:
 -13.831122
STAR:
 274.694398
STAR:
 -13.829145
STAR:
 274.694578
STAR:
 -13.827611
STAR:
 274.688823
STAR:
 -13.816141
STAR:
 274.689728
STAR:
 -13.814786
STAR:
 274.691117
STAR:
 -13.824872
STAR:
 274.694991
STAR:
 -13.819376
STAR:
 274.693014
STAR:
 -13.824153
STAR:
 274.692207
STAR:
 -13.823181
STAR:
 274.691538
STAR:
 -13.8

In [ ]:
# Display the image with the location of the star above circled

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection=wcs)

Drawing_colored_circle = plt.Circle(( pixel_coords[0] , pixel_coords[1] ), 3, fill=False, edgecolor='Blue')
ax.add_artist( Drawing_colored_circle )
from 
ax.imshow(image_hdu.data, cmap='gray', origin='lower')
ax.set_title(f'{object_name}')
ax.set_xlabel('RA')
ax.set_ylabel('Dec')
ax.grid(color='white', ls='dotted')

In [19]:
# # Fetch star data from Vizier using the Hipparcos catalog
# v = Vizier(columns=['*'])
# catalog_list = v.query_region(coords, radius=2 * u.deg, catalog='I/239/hip_main')
# catalog = catalog_list[0]

# # Convert the table to a pandas DataFrame for easier manipulation
# catalog = catalog.to_pandas()
# print(catalog)

# # Save the catalog as a CSV file
# catalog.to_csv(f'images/{object_name}/Hipparcos_catalog_{object_name}.csv', index=False)

In [39]:
# # Example preprocessing steps
# # Normalize the image
# normalized_image = (image - np.min(image)) / (np.max(image) - np.min(image))

In [40]:
# # Extract star positions and magnitudes
# star_positions = np.array([stars['RAJ2000'], stars['DEJ2000']]).T
# star_magnitudes = stars['Vmag']

In [41]:
# # Save preprocessed data for further analysis
# np.save('preprocessed_image.npy', normalized_image)
# star_data = pd.DataFrame({'RA': stars['RAJ2000'], 'Dec': stars['DEJ2000'], 'Magnitude': stars['Vmag']})
# star_data.to_csv('preprocessed_star_data.csv', index=False)

In [42]:
# # Save the image using OpenCV
# cv2.imwrite('sky_image.png', image)